In [ ]:
#!/usr/bin/env python
# coding: utf-8

# creates a basis of a specified dimension that obeys a hadamard bound
def create_good_basis(n,B,hadamard_bound):
    while True:
        basis = random_matrix(ZZ, n, x=-B, y=B) #generate a random basis from Sage
        if hadamard_ratio(basis,n) > hadamard_bound:
            return basis

In [ ]:

# calculate the hadamard ratio of a basis of dimension n
def hadamard_ratio(L,n): 
    ratio = abs(det(matrix((L))))
    for v in L:
        ratio = ratio/norm(vector(v))
    return ratio**(1/n)
    

In [ ]:

# applies a change of basis matrix to a given good basis
def create_bad_basis(n, b_priv,B, num_ops):
    b_priv = matrix(b_priv)
    COB = create_COB(n,num_ops,B)
    return COB*b_priv


# creates a change of basis matrix by performing random elementary row operations on an identity matrix of dimension n
def create_COB(n,num_ops,B):
    id = identity_matrix(ZZ,n)
    for x in range(num_ops):
        i = randrange(n)
        j = randrange(n)
        if i != j:
            id[i] = id[i] + id[j] * randrange(-B,B)
    return id 


In [ ]:

# babai's algorithm that approx. solves the cvp by rounding the linear coordinates of the given vector in the given basis. uses a helper function round_coords to round each entry. 
def babai(basis, vector):
    t = express_v_in_basis(basis,vector)
    t = list(list(t)[0])
    
    tr = round_coords(t)
    tr = matrix(tr)
   
    vectorp = tr * matrix(basis)
    return list(list(vectorp)[0]) #returns the solution to the cvp

def round_coords(w):
    return [round(ti) for ti in w]


#helper function to subtract vectors
def subtract_vectors(v1, v2):
    v1 = matrix(v1)
    v2 = matrix(v2)
    return [v1[0, i] - v2[0, i] for i in range(len(v1[0]))]
    


# helper function that returns the linear coordinates of a vector expressed in a given basis by multiplying the vector by the inverse matrix
def express_v_in_basis(basis,vector):
    bm = matrix(basis)
    vm = matrix([vector])
    return vm * (bm^(-1))


In [ ]:
# samantha generates a private, good basis 
def samantha_private_key_creation(n,B,hadamard_bound):
    b_priv = create_good_basis(n,B,hadamard_bound)
    return b_priv



# samantha generates a public, bad basis using a helper function
def samantha_public_key_creation(b_priv,n,B,num_ops):
    return create_bad_basis(n,b_priv,B,num_ops)



# samantha generates a bad public basis and uses Babai to approx. solve the cvp using her private key for document d. returns the bad public basis and expression of solution s to cvp in terms of bad basis (d_sig)
def samantha_sign(b_priv,d,b_pub):
    s = babai(b_priv, d)
    d_sig = express_v_in_basis(b_pub,s)
    return d_sig


In [ ]:
# victor verifies that the signature is valid by checking that the distance between d and d_sig obeys a given bound
def victor_verify(b_pub, d_sig, d, distance_bound): 
    d_sig = matrix(d_sig)
    b_pub = matrix(b_pub)
    s = d_sig * b_pub
    s_minus_d = matrix(subtract_vectors(s,d))
    distance = s_minus_d.norm()
    if distance < distance_bound:
        return True, distance
    else:
        return False, distance


In [ ]:

#function putting everything together. takes in a pre-determined b_priv and b_pub for testing purposes.

def GGH_sign(document,dimension,distance_bound,b_priv,b_pub):
    d_sig = samantha_sign(b_priv,document,b_pub)
    valid, distance = victor_verify(b_pub, d_sig, document, distance_bound)[0], victor_verify(b_pub, d_sig, document, distance_bound)[1]
    if valid:
        print(d_sig, 'is a valid signature for ', document, 'and the distance between vectors is', distance)
    else:
        print('Looks like your signature is invalid! The distance between vectors was:', distance)

In [ ]:
#2-d example from paper
distance_bound = 100

v1 = [84,58]
v2 = [3,-68]
b_priv = [v1,v2]

v1 = [-363885,-243476]
v2 = [3279,2194]
b_pub = [v1, v2]

d = [123,456]

GGH_sign(d, len(d), distance_bound, b_priv, b_pub)


In [ ]:
# To run examples, provide public parameters:

hadamard_bound = x
B = x #range for integer values of entries in b_priv
n = x #dimension
num_ops = x #number of row operations to make when generating the change of basis matrix
d = [x] #the document vector to be signed
distance_bound = x #the upper bound for a signature to be valid on d 

b_priv = samantha_private_key_creation(n,B,hadamard_bound)
print("b_priv is:", b_priv)
b_pub = samantha_public_key_creation(b_priv,n,B,num_ops)
print("b_pub is:", b_pub)

d_sig = samantha_sign(b_priv,d,b_pub)
print("d_sig is:", d_sig)

valid,distance = victor_verify(b_pub, d_sig, d, distance_bound)[0], victor_verify(b_pub, d_sig, d, distance_bound)[1]
print("The signature was ", valid, " and the distance between vectors was ", distance)


In [1]:
# the following cells are the code for rejection sampling built into GGH. they require hashlib and random imports. 

import hashlib
import random

# assumes the d passed in is already in vector form
def hash_document(d):
    vector_string = ''.join(str(element) for element in d) #turn every element of d into a string and concatenate together
    vector_string = randomize_d(vector_string)
    vector_string = vector_string.encode() #encodes string into necessary format for SHA256
    hashGen = hashlib.sha512()
    hashGen.update(vector_string)
    hash = hashGen.hexdigest() #this is a fixed length of 128 characters (512 bit) hexidecimal string

    return hash

# coverts the hash into a vector so that we can perform lattice operations on it
def hash_to_vector(hashed_d, d, num_bytes):
    m = 2*num_bytes
    dimension = len(d)

    if 128 > m*dimension: #hash too long, truncate
        hashed_d = hashed_d[:m*dimension]
    else: #hash too short, extend
        bytes_to_add = 128 - m*dimension
        bits_to_add = (8*m)*bytes_to_add
        random_hexadecimal = '%0x' % random.getrandbits(bits_to_add)
        hashed_d = random_hexadecimal + hashed_d
    
    hashed_vector = [int(hashed_d[i:i+m], 16) for i in range(0, len(hashed_d), m)]
  
    return hashed_vector
    
# generates a random number to concatenate to d to randomize 1
def randomize_d(d):
    num = random.randint(int(d),int(d)**4)
    d = d + str(num)
    return d


In [2]:
# modified signing so that it requires that s-d falls into the common region defined by the rejection_sample_bound (side length of common box)
def modified_samantha_sign(b_priv,b_pub , d,rejection_sample_bound,num_bytes):
    count = 0
    while True:  
        hashed_d = hash_document(d)
        hashed_vector = hash_to_vector(hashed_d, d,num_bytes)
        s = babai(b_priv, hashed_vector)
        s_minus_d = subtract_vectors(s,hashed_vector)
        if check_rejection_sample(s_minus_d,rejection_sample_bound):
            d_sig = express_v_in_basis(b_pub,s)
            print("Tried ", count, "signatures before finding one.")
            return d_sig, hashed_vector
        count = count + 1



# checks whether each entry in vector s-d is within the bound or not
def check_rejection_sample(s_minus_d, bound):
    for entry in s_minus_d:
        if abs(entry) > 0.5*bound:
            return False # returns false if any of the entries do not fall in region
    return True



In [3]:
#rejection sampling using the 2-d example in paper

v1 = [84, 58]
v2 = [3,  -68]
b_priv = [v1,v2]

v1 = [-363885,-243476]
v2 = [3279,2194]
b_pub = [v1, v2]

d = [123,456]

rejection_bound = 1

num_bytes = 2 # values 0 to 65,535

d_sig,hashed_vector = modified_samantha_sign(b_priv,b_pub, d, rejection_bound, num_bytes)

print("The signature that met the condition was ", d_sig, "and the hashed version of the document vector used was ", hashed_vector)


<class 'NameError'>: name 'babai' is not defined

In [ ]:

# To try rejection sampling, provide parameters: (note that i couldnt't get it to run fast enough for dimensions higher than ~4. choose high rejection bounds (100+) if you want it to return)

b_priv = [x] # can use create_good_basis(n,B,hadamard_bound)
b_pub = [x]# can use create_bad_basis(n,b_priv,B,num_ops)
d = [x] #document vector
rejection_sample_bound = x
num_bytes = x # number of bytes you want in each entry of the vector

d_sig,hashed_vector = modified_samantha_sign(b_priv, b_pub, d, rejection_sample_bound, num_bytes)

print("The signature that met the condition was ", d_sig, "and the hashed version of the document vector used was ", hashed_vector)


